In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import scipy.stats as stats

In [ ]:
'''
In-sample: beta & sample-mean are all known;
Out-of-sample: beta is known, sample-mean is unknown;

k = 10
Experiment 0:
Normal distribution

Experiment 1:
beta in range(0, 1, 0.2) -> 0, 0.2, 0.4, 0.6, 0.8, 1; lambda = 0.5; box, elliposoid -> return
lambda in range(0, 1, 0.2) -> 0, 0.2, 0.4, 0.6, 0.8, 1; beta = 0.5; box, elliposoid -> return 
20240102,   1.47,  -0.34,  -1.15,   1.07,   0.40,  -2.21,   1.21,   1.29,  -0.70,   1.49,   0.41,  -0.75

Experiment 2:
in-sample vs out-of-sample
LSMT -> beta vs Transformer -> beta, box, elliposoid -> return

Dataset Divide:
In-Sample: 2512 data points & labels -> trained model; 2512 + trained model -> beta

Out-of-Sample: 2512 divided into two parts: 2000 data points -> training model; 512 data points -> testing model

ML:
n-k~n-th stock price = [12*1] -> n+k-th S&P 500 Index -> beta;
k = 2
Day:   1  2  3  4  5  6  7  8; 
Index: 1  1  3  4  2  4  5  6;
Label: N  N  0  1  0  0  1  1;
beta = 3/6 = 0.5

N days 
ML(stock price) -> Label;

Historical Data:
N days * M stocks -> mean price;
max w u
s.t w1 = 1
Porfolio Strategy:
-> w -> 
Future Data:
K days * M stocks -> returns;

RO -> u;

ML -> beta;
ML(t) -> label(t+k) k = 5
k = 3
Previous Data:
1 2 3 4 5 

1 2 3 -> 3
2 3 4 -> 4
3 4 5 -> 5

Future Data:
6 7 8 9 10 
1 0 1 0 0

beta = 2 / 5 = 0.4


'''

'\nExperiment 0:\nNormal distribution\n\nExperiment 1:\nbeta in range(0, 1, 0.2) -> 0, 0.2, 0.4, 0.6, 0.8, 1; lambda = 0.5; box, elliposoid -> return\nlambda in range(0, 1, 0.2) -> 0, 0.2, 0.4, 0.6, 0.8, 1; beta = 0.5; box, elliposoid -> return \n20240102,   1.47,  -0.34,  -1.15,   1.07,   0.40,  -2.21,   1.21,   1.29,  -0.70,   1.49,   0.41,  -0.75\n\nExperiment 2:\nin-sample vs out-of-sample\nLSMT -> beta vs Transformer -> beta, box, elliposoid -> return\n\nDataset Divide:\nIn-Sample: 2512 data points & labels -> training model; 30/2512 -> testing model\n\nOut-of-Sample: 2512 divided into two parts: 2000 data points -> training model; 512 data points -> testing model\n\nML:\n'

In [2]:
import pickle
import os

stocks = pickle.load(open(os.path.dirname(os.getcwd())+'/Prepare_Datasets/indicators.pkl', 'rb'))
samples = list()
for i in stocks:
    samples.append(stocks[i])
samples = np.asarray(samples, dtype=np.float64)
print(samples)

[[-1.03 -0.66 -0.99 ... -0.39 -0.63 -1.03]
 [-0.11 -0.48  0.33 ...  0.23  0.56  0.1 ]
 [-0.51  0.02 -0.56 ... -0.39 -0.05 -0.9 ]
 ...
 [ 0.32  1.33  0.06 ...  0.67  0.33  0.15]
 [ 0.22 -2.24 -0.11 ...  0.17  0.33  0.06]
 [ 0.06 -1.49 -0.38 ... -0.18 -0.39 -0.55]]


In [4]:
# N assets
N = 10
# S sample
S = 100
# Sigma
Sig = np.eye(N)
# Sigma^{1/2}
Sig2 = np.linalg.cholesky(Sig)
print(np.allclose(Sig, Sig2 @ Sig2.T)) 
# muhat
mu = np.zeros((N,1))
#confidence level alpha
alpha = 0.05
# standard norm quantile
z_95 = stats.norm.ppf(1-alpha)
z_975 = stats.norm.ppf(1-alpha/2)
# delta^2 = chi^2 quantile
delta2 = stats.chi2.ppf(1-alpha, df=N)
# lambda0
lambda0 = 0.5
# beta
beta = 0.5


True


In [5]:
# Box uncertainty set
A = np.hstack((Sig2, np.zeros((N,1))))
c = mu + (1-2*beta)*(z_975/np.sqrt(S))*Sig2@np.ones((N,1))
c = np.append(c, 1)
c = c/((1-lambda0)*z_95)
e = np.ones(N+1)
e[-1] = 0

In [20]:
# set random seed
np.random.seed(0)
# objective function coeficient
f = np.zeros(N + 1)  
f[-1] = 1      

# define variables x=[w,t]^T
x = cp.Variable(N+1)
# SOC constraints
soc_constraints = [cp.SOC(c.T @ x , A @ x )]
# Define and solve the CVXPY problem.
prob = cp.Problem(cp.Minimize(f.T@x),
                  soc_constraints + [e.T @ x == 1])

prob.solve()
# Print result.
print("The optimal value:", prob.value)
print("Optimal solution x:", x.value)

The optimal value: 0.2600741922463065
Optimal solution x: [0.1        0.1        0.1        0.1        0.1        0.1
 0.1        0.1        0.1        0.1        0.26007419]


In [26]:
A1 = np.hstack((Sig2, np.zeros((N,1))))
c1 = np.append(mu, 1)
a = (1-lambda0)*z_95 + (1-2*beta)*np.sqrt(delta2/S)
c1 = c1/a
e1 = np.ones(N+1)
e1[-1] = 0
print(e1)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


In [28]:
# set random seed
np.random.seed(0)
# objective function coeficient
f1 = np.zeros(N + 1) 
f1[-1] = 1      
print(f1.shape)

# define variables x=[w,t]^T
x1 = cp.Variable(N+1)
# SOC constraints
soc_constraints1 = [cp.SOC(c1.T @ x1 , A1 @ x1 )]
# Define and solve the CVXPY problem.
prob1 = cp.Problem(cp.Minimize(f1.T@x1),
                  soc_constraints1 + [e1.T @ x1 == 1])

prob1.solve()
# Print result.
print("The optimal value:", prob1.value)
print("Optimal solution x:", x1.value)

(11,)
The optimal value: 0.2600741922463065
Optimal solution x: [0.1        0.1        0.1        0.1        0.1        0.1
 0.1        0.1        0.1        0.1        0.26007419]
